In [69]:
import requests
import json
from dotenv import load_dotenv
import os
from web3 import Web3
load_dotenv()  # have your rpc endpoint in .env file to prevent accidental commit to github

to_address = '0x50f9202e0f1c1577822bd67193960b213cd2f331' # hacker EOA
contract_address = '9eb5f8e83359bb5013f3d8eee60bdce5654e8881' # hacker contract
approve_method_id = '095ea7b3'
url = "https://q.luabase.com/run"

# connect to the RPC endpoint
rpc_endpoint_address=os.environ.get('ETHEREUM_RPC_ENDPOINT')
w3 = Web3(Web3.HTTPProvider(rpc_endpoint_address))

def queryLuabase(sql):
    headers = {"content-type": "application/json"}
    payload = {
        "block": {
            "details": {
            "sql": sql,
            }
        },
        "api_key": os.environ.get('LUABASE_API_KEY')
    }

    res = requests.request("POST", url, json=payload, headers=headers).json()
    if res['data'] is not None:
        return res['data']
    return res

In [70]:
# collect list of erc20 tranfer txs that were sent to the hacker's address
transfer_data = queryLuabase(f"select from_address as victim_from_address,token_address as erc20_token_address,to_address as hacker_to_address,transaction_hash, value as amount, block_number FROM ethereum.token_transfers AS t WHERE t.to_address = '{to_address}' AND t.block_number >= 15213261")
len(transfer_data)


11

In [71]:
# collect list of txs that have the withdraw erc20 balance method id of victim's addresses & the address called is the hacker's contract. 
transaction_data = queryLuabase(f"select from_address,to_address,hash,block_number FROM ethereum.transactions AS t where t.input LIKE '%0x9c307de6%' AND t.to_address = '0x{contract_address}'")
len(transaction_data)

9

In [72]:
# collect victim's addresses
final_data = []
for t in transfer_data:
    for tx in transaction_data:
        if t['transaction_hash'] == tx['hash']:
            # print(tx)
            t['hacker_contract_address'] = tx['to_address']
            final_data.append(t)

print(len(final_data))
for data in final_data:
    print(data['victim_from_address'])

9
0x45470961fb25789654fcfc391baf77019051ef78
0x986680f665eb6fb6f72c9df209468e3426517bc6
0x1a2a910185467dab98c3c04eee38ac6dbbc3b0d5
0xbf87bfcaf948c853cb4edbf07cb8dab80319fc5e
0x8c35933c469406c8899882f5c2119649cd5b617f
0xb30ca7383dbd1ac31459e8e1de3bcb022a1db001
0x056fc75753bd75ea28b5966c4bea29f4ad49321c
0x4436f5dd7bc4a63bcd86a33ba2862fdbf242c76e
0xb97f255b37eef7d47dbe1debbecd1f2ecfc37abe


In [73]:
# get erc20 information to calcuate the total assets stolen. 
unique_erc20s = {}

assets = []
for f in final_data:
    erc20 = f['erc20_token_address']
    if erc20 not in unique_erc20s:
        response = queryLuabase(f"select symbol, decimals from ethereum.tokens as t WHERE t.address = '{erc20}'")[0]
        unique_erc20s[erc20] = {'erc20_decimals': response['decimals'], 'erc20_symbol': response['symbol']}
    f.update(unique_erc20s[erc20])

final_data[0]

{'victim_from_address': '0x45470961fb25789654fcfc391baf77019051ef78',
 'erc20_token_address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
 'hacker_to_address': '0x50f9202e0f1c1577822bd67193960b213cd2f331',
 'transaction_hash': '0x3fb7c60fea183a77bf548858cbb1d77c2a824373822bb6bbb32e69c1d2188a1c',
 'amount': 16880678,
 'block_number': 15309725,
 'hacker_contract_address': '0x9eb5f8e83359bb5013f3d8eee60bdce5654e8881',
 'erc20_decimals': 6,
 'erc20_symbol': 'USDC'}

In [75]:
# calculate total assets stolen based on the ERC20 information & transaction data retrieved

asset_lost_results = {}

for f in final_data:
    erc20 = f['erc20_token_address']
    if erc20 not in asset_lost_results:
        asset_lost_results[erc20] = {'amount': f['amount'], 'decimals': f['erc20_decimals'], 'symbol': f['erc20_symbol']}
    else:
        asset_lost_results[erc20].update({'amount': f['amount'] + asset_lost_results[erc20]['amount']})

for asset in asset_lost_results.keys():
    asset_lost_results[asset]['amount'] = asset_lost_results[asset]['amount'] / pow(10, asset_lost_results[asset]['decimals'])

asset_lost_results

{'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': {'amount': 606265.325268,
  'decimals': 6,
  'symbol': 'USDC'},
 '0x6b175474e89094c44da98b954eedeac495271d0f': {'amount': 6458.823763845549,
  'decimals': 18,
  'symbol': 'DAI'}}

In [ ]:
# query list of txs that hacker cashes out (eth to other platform). value_eth >= 0.1 based on etherscan
response = queryLuabase(f"select block_number, hash, to_address, value_eth, tag from ethereum.transactions as t left join ethereum.tags as tags on t.to_address = tags.address WHERE t.value_eth >= 0.1 and t.from_address = '{to_address}'")
response

            

[{'block_number': 15310366,
  'hash': '0xd917bc4ae6207e0aad066eae5fd309c077b94aef2b1953c659f6faec2aa340cf',
  'to_address': '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b',
  'value_eth': 1.0,
  'tag': 'Tornado.Cash: Router'},
 {'block_number': 15310377,
  'hash': '0xd118f1db41f3f46835d223511406c340b41f1f50cf741e0b1ec9945d68db5a64',
  'to_address': '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b',
  'value_eth': 0.1,
  'tag': 'Tornado.Cash: Router'},
 {'block_number': 15310380,
  'hash': '0xd98aa6edaaf61a5e14f2c7545a6d887acc467c9db1a4f5a7e30b0afcded7630c',
  'to_address': '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b',
  'value_eth': 0.1,
  'tag': 'Tornado.Cash: Router'},
 {'block_number': 15310409,
  'hash': '0xd127c04c25adada5de8817e4e577ba71d15aa1a11bd2f1100ef59afd16f53375',
  'to_address': '0xd90e2f925da726b50c4ed8d0fb90ad053324f31b',
  'value_eth': 0.1,
  'tag': 'Tornado.Cash: Router'},
 {'block_number': 15309715,
  'hash': '0xeaebd96cd99db2f27165d761f6d69c0222714214678c47541827d63a6a0f535

In [ ]:
# filter cash out based on tag

cash_out_only_tags = []
cash_out_known_tags = []
cash_out_unknown_tags = []

for r in response:
    reduced_r = {'value_eth':r['value_eth'], 'tag': r['tag']}
    if r['tag'] == '':
        cash_out_unknown_tags.append(r)
        continue

    if r['tag'] not in cash_out_only_tags:
        cash_out_only_tags.append(r['tag'])
        cash_out_known_tags.append(reduced_r)
        continue

    for c in cash_out_known_tags:
        if r['tag'] == c['tag']:
            c['value_eth'] += r['value_eth']
            break

print(cash_out_known_tags)
print(cash_out_unknown_tags)

[{'value_eth': 27.700000000000003, 'tag': 'Tornado.Cash: Router'}]
[{'block_number': 15309715, 'hash': '0xeaebd96cd99db2f27165d761f6d69c0222714214678c47541827d63a6a0f5359', 'to_address': '0x45470961fb25789654fcfc391baf77019051ef78', 'value_eth': 0.1, 'tag': ''}, {'block_number': 15310223, 'hash': '0x2e76259606c3f4fc2842e1dce99c2631753c4526b7c615312f2a65b285689d48', 'to_address': '0x06c339416d76fce9ca8a7a66b4aedbe7301826d7', 'value_eth': 45.0, 'tag': ''}, {'block_number': 15310242, 'hash': '0x5108cd31c7a16727275d2f55fe2a31f2f3f20cd3821c485b1917d9411c97fb70', 'to_address': '0xc3e20d299084b7017ad44e6cd2ba0a1d39ab2e31', 'value_eth': 45.0, 'tag': ''}, {'block_number': 15310252, 'hash': '0x069620d1f2b81edfa8f6ff25e3f45f53e43378e7fc9f637f7d227ed8e27fbec8', 'to_address': '0xc137c19946724f76e65bb0efb81ddcf895d6472c', 'value_eth': 45.0, 'tag': ''}, {'block_number': 15310290, 'hash': '0x57da7f38aa453f08804de988bb093c6e359a90c3e753609f9d29e26d78225e30', 'to_address': '0xb6d318c4d1618eb8dcc2e8c0960

In [ ]:
# collect cash out txs for unknown tags. These addresses are also from hacker's
# reuse var
cash_out_new_known_tags = []
for c in cash_out_unknown_tags:
    response = queryLuabase(f"select block_number, hash, to_address, value_eth, tag from ethereum.transactions as t left join ethereum.tags as tags on t.to_address = tags.address WHERE t.value_eth >= 0.1 and t.from_address = '{c['to_address']}'")
    cash_out_new_known_tags.append(response)

cash_out_new_known_tags

[[],
 [{'block_number': 15310245,
   'hash': '0x5cbe818b9bbfa4344e48c8bb5d37c0eecdcde8b15613c6823ff13d0af72c353d',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 44.99972700000001,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310300,
   'hash': '0x22df8dee167bbf32bfecca2aa1aa33487402a3c840383f0b9b4dced215553d54',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 44.999706,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310274,
   'hash': '0x64fdd3998e60b98058d524dd5c9ea7c22db039a276e3594501fd18aee9f08bd4',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 44.999391,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310304,
   'hash': '0x81df74a39265d6f85e19ec0954c90e393d6df4bf7f250ddf0c31dfe580d1dbb1',
   'to_address': '0x4e5b2e1dc63f6b91cb6cd759936495434c7e972f',
   'value_eth': 21.999643,
   'tag': 'FixedFloat'}],
 [{'block_number': 15310229,
   'hash': '0x2eef64d081a222a9d1667d27caf76a1034eba96c

In [ ]:
# continue to filter more tags
tags = []
unknown_tags = []
only_tags = []

for c in cash_out_new_known_tags:
    for cc in c:
        reduced_c = {'value_eth':cc['value_eth'], 'tag': cc['tag']}
        if cc['tag'] == '':
            unknown_tags.append(reduced_c)
            continue

        if cc['tag'] not in only_tags:
            only_tags.append(cc['tag'])
            tags.append(reduced_c)
            continue

        for t in tags:
            if cc['tag'] == t['tag']:
                t['value_eth'] += cc['value_eth']
                break

unknown_tags[0]['tag'] = 'SideShift: Hot Wallet'
print(tags)
print(cash_out_known_tags)
print(unknown_tags)

[{'value_eth': 291.99741700000004, 'tag': 'FixedFloat'}, {'value_eth': 19.997937, 'tag': 'Binance 14'}]
[{'value_eth': 27.700000000000003, 'tag': 'Tornado.Cash: Router'}]
[{'value_eth': 22.999450053588944, 'tag': 'SideShift: Hot Wallet'}]


In [80]:
abi='[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"authorizer","type":"address"},{"indexed":true,"internalType":"bytes32","name":"nonce","type":"bytes32"}],"name":"AuthorizationCanceled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"authorizer","type":"address"},{"indexed":true,"internalType":"bytes32","name":"nonce","type":"bytes32"}],"name":"AuthorizationUsed","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_account","type":"address"}],"name":"Blacklisted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newBlacklister","type":"address"}],"name":"BlacklisterChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"burner","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newMasterMinter","type":"address"}],"name":"MasterMinterChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"minter","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"minter","type":"address"},{"indexed":false,"internalType":"uint256","name":"minterAllowedAmount","type":"uint256"}],"name":"MinterConfigured","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"oldMinter","type":"address"}],"name":"MinterRemoved","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":false,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[],"name":"Pause","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newAddress","type":"address"}],"name":"PauserChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newRescuer","type":"address"}],"name":"RescuerChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_account","type":"address"}],"name":"UnBlacklisted","type":"event"},{"anonymous":false,"inputs":[],"name":"Unpause","type":"event"},{"inputs":[],"name":"CANCEL_AUTHORIZATION_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RECEIVE_WITH_AUTHORIZATION_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"TRANSFER_WITH_AUTHORIZATION_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"authorizer","type":"address"},{"internalType":"bytes32","name":"nonce","type":"bytes32"}],"name":"authorizationState","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_account","type":"address"}],"name":"blacklist","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"blacklister","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"burn","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"authorizer","type":"address"},{"internalType":"bytes32","name":"nonce","type":"bytes32"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"cancelAuthorization","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"minter","type":"address"},{"internalType":"uint256","name":"minterAllowedAmount","type":"uint256"}],"name":"configureMinter","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"currency","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"decrement","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"increment","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"string","name":"tokenName","type":"string"},{"internalType":"string","name":"tokenSymbol","type":"string"},{"internalType":"string","name":"tokenCurrency","type":"string"},{"internalType":"uint8","name":"tokenDecimals","type":"uint8"},{"internalType":"address","name":"newMasterMinter","type":"address"},{"internalType":"address","name":"newPauser","type":"address"},{"internalType":"address","name":"newBlacklister","type":"address"},{"internalType":"address","name":"newOwner","type":"address"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"string","name":"newName","type":"string"}],"name":"initializeV2","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"lostAndFound","type":"address"}],"name":"initializeV2_1","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_account","type":"address"}],"name":"isBlacklisted","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"isMinter","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"masterMinter","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"mint","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"minter","type":"address"}],"name":"minterAllowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pauser","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"validAfter","type":"uint256"},{"internalType":"uint256","name":"validBefore","type":"uint256"},{"internalType":"bytes32","name":"nonce","type":"bytes32"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"receiveWithAuthorization","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"minter","type":"address"}],"name":"removeMinter","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"tokenContract","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"rescueERC20","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"rescuer","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"validAfter","type":"uint256"},{"internalType":"uint256","name":"validBefore","type":"uint256"},{"internalType":"bytes32","name":"nonce","type":"bytes32"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"transferWithAuthorization","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_account","type":"address"}],"name":"unBlacklist","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"unpause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_newBlacklister","type":"address"}],"name":"updateBlacklister","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_newMasterMinter","type":"address"}],"name":"updateMasterMinter","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_newPauser","type":"address"}],"name":"updatePauser","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newRescuer","type":"address"}],"name":"updateRescuer","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"}]'

contract = w3.eth.contract(Web3.toChecksumAddress('0x6b175474e89094c44da98b954eedeac495271d0f'), abi=abi)
raw_balance = contract.functions.balanceOf(Web3.toChecksumAddress('0x6b175474e89094c44da98b954eedeac495271d0f')).call(block_identifier=14000000)
raw_balance

ValueError: {'code': -32043, 'message': 'Requested data cannot be older than 128 blocks.'}